In [2]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [15]:
%%writefile face_mask_detection.py
import streamlit as st
import cv2
import numpy as np
from tensorflow.keras.models import load_model

#load h5 model
model = load_model('/content/face_mask_detection_model.h5')

st.title('Face Mask Detection Application')

uploaded_image = st.file_uploader("upload an image...", type= ['jpg', 'png', 'jpeg'])

#define a function to detect faces
def detect_and_predict_mask(image):
  face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
  faces = face_cascade.detectMultiScale(image, scaleFactor=1.1)

  predictions = []

  for (x,y,w,h) in faces:
    face = image[y:y+h, x:x+w]
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face = cv2.resize(face, (128,128))
    face = np.array(face) / 255.0
    face = np.expand_dims(face, axis = 0)

    #predict mask / no mask
    predictions.append(model.predict(face))

  return faces, predictions

if uploaded_image is  not None:
  file_bytes = np.asarray(bytearray(uploaded_image.read()), dtype= np.uint8)
  image = cv2.imdecode(file_bytes,1)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  st.image(image)

  faces, predictions = detect_and_predict_mask(image)

  for i , (x,y,w,h) in enumerate(faces):
    (mask, withoutMask) = predictions[i][0]
    label = 'Mask' if mask > withoutMask else 'No Mask'
    color = (0,0,255) if label == 'Mask' else (255,0,0)

    cv2.putText(image, label, (x,y -10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.rectangle(image, (x,y), (x+w, y+h), color, 1)

  st.image(image, caption = 'Detected Image')

Overwriting face_mask_detection.py


In [4]:
#setup ngrok with your authonken
!ngrok authtoken 2JQ7xQIMC37Y473rZ6ZbapzlPMB_4cZK6X28eFyAPsEfKCwD8

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
from pyngrok import ngrok
!streamlit run face_mask_detection.py &>/dev/null&

In [6]:
!pgrep -f streamlit

8902


In [7]:
public_url = ngrok.connect(8501)
public_url

<NgrokTunnel: "https://ad75-34-48-139-214.ngrok-free.app" -> "http://localhost:8501">

In [16]:
ngrok.kill()